In [ ]:
# default_exp TaskTemplate

In [ ]:
# export
from pathlib import Path
from nbdev_exp.HeteroData_synthesize import SyntheticDataModule, HeteroGNNArch, RamSimpleModel
from clearml import Task
import pytorch_lightning as pl

TASK_NAME = "HP searching"
MY_PROJECT_NAME = "MY_PROJECT_NAME"


In [ ]:
#export
def my_first_task_template(task_template_name=TASK_NAME,**kwargs):
    task = Task.init(project_name=MY_PROJECT_NAME, task_name=task_template_name)

    args = {
        "max_epochs":35,

        "batch_size": 100,
        "hidden_channels": 10,
        "num_layers": 2,
    }
    args = task.connect(args)

    synthetic_data_module = SyntheticDataModule(**args)

    model_arch = HeteroGNNArch(**args)

    model = RamSimpleModel(model_arch=model_arch)
    model.lazy_initialize(synthetic_data_module.get_single_sample())
    model.set_static_edge_index(synthetic_data_module.static_edge_index_dict)

    logs_root =f"{str((Path.cwd().parent / 'logs').absolute())}"
    tb_logger = pl.loggers.tensorboard.TensorBoardLogger(save_dir=logs_root, name=task_template_name)
    trainer = pl.Trainer(logger=tb_logger,max_epochs=args["max_epochs"])
    trainer.fit(model, synthetic_data_module)
    task.mark_complete()

In [ ]:
my_first_task_template()